## 知识库的构建
### 文档解析-->分块-->embedding-->向量数据库

In [3]:
import pandas as pd
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
from model import RagEmbedding, RagLLM, QwenLLM
from tools.doc_parse import chunk, read_and_process_excel, logger

In [4]:
# 加载文档集
pdf_files = ['./data/zhidu/zhidu_employee.pdf',
            './data/zhidu/zhidu_travel.pdf']

excel_files = ['./data/zhidu/zhidu_detail.xlsx']

In [7]:
# 实例化递归文本分割对象
r_spliter = RecursiveCharacterTextSplitter(chunk_size=128,
                                          chunk_overlap=30,
                                          separators=["\n\n",
                                                      "\n",
                                                      ".",
                                                      "\uff0e",  # Fullwidth full stop
                                                       "\u3002",  # Ideographic full stop
                                                      ",",
                                                      "\uff0c",  # Fullwidth comma
                                                      "\u3001",  # Ideographic comma
                                                     ])


In [ ]:
# 解析并切割pdf内容
doc_data = []
for pdf_file_name in pdf_files:
    res = chunk(pdf_file_name, callback=logger)
    for data in res:
        content = data["content_with_weight"]
        if '<table>' not in content and len(content) > 200:
            # 超过200字符，使用递归文本对象再次分块
            doc_data = doc_data + r_spliter.split_text(content)
        else:
            doc_data.append(content)

for i in doc_data:
    print(len(i), "="*10,  i)


In [2]:
# 处理excel
for excel_file_name in excel_files:
    data = read_and_process_excel(excel_file_name)
    df = pd.DataFrame(data[8:], columns=data[7])
    data_excel = df.drop(columns=df.columns[11:17]) # 去掉空列
    content = data_excel.to_markdown(index=False).replace(' ', "")
    doc_data.append(f"### 以下是中央和国家机关工作人员赴地方差旅住宿费标准明细表： \n\n {content}")

NameError: name 'excel_files' is not defined

In [ ]:
# 封装成document，以支持向量化
from langchain_core.documents import Document
import uuid
documents = []
document_ids = []
for idx, chunk in enumerate(doc_data):
    is_table = 0
    if "table" in chunk:
        is_table = 1
    if idx == len(doc_data) - 1:
        is_table = 1
    doc_id = str(uuid.uuid4())
    document = Document(
        page_content=chunk,
        metadata={"type": "ori", "is_table": is_table})
    documents.append(document)
    document_ids.append(doc_id)





d:\Desktop\LLM资料\Proj\ChatDoc_RagLangChain\ChatDoc1.0\model.py:97: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedding = HuggingFaceEmbeddings(model_name=model_path,


In [ ]:
# 向量化处理
embedding_cls = RagEmbedding(model_path="./data/llm_app/embedding_models/bge-m3//")

In [ ]:
# 用于pickle化
doc_txts = dict(zip(document_ids, documents))

import pickle
with open('./data/zhidu/zhidu_db.pickl', 'wb') as file:
    pickle.dump(doc_txts, file)



In [ ]:
# 与Chroma向量库建立连接
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

# 创建向量库对象
embedding_db = Chroma.from_documents(documents,
                                     embedding_cls.get_embedding_fun(),
                                     client=chroma_client,
                                     collection_name="zhidu_db",
                                     )

NameError: name 'chromadb' is not defined

In [ ]:
# 测试向量搜索
query = '迟到有什么规定？'

# 找到相似度最高的前k个文本块
related_docs = embedding_db.similarity_search(query, k=2)


print("related_docs:" ,related_docs)